## Sample code for connecting from external JupyterHub to CAS using the SWAT package for Python. 

You will need to install the SAS SWAT package.  Documentation how to do that can be found here:
* https://github.com/sassoftware/python-swat/releases
* https://sassoftware.github.io/python-swat/install.html

In [1]:
import swat
import os

There are two ways of connecting to CAS: 

* using REST API calls or 
* using a SAS-native binary protocol.

The binary protocol is faster but not supported on all platforms.   Here we're using <team>-cas.hackinsas.vectorlabs.sas.com as CAS hostname. We can do that because both ends (CAS and Jupyter) are running as pods inside the Kubernetes cluster. 
    
On Windows and ios clients you need to use REST.  See the 2nd example in the cell below.  This time we use the same hostname as we use to connect to other Viya user interfaces.
    
**Important**    
Please note you have to upload the correct CA certificate as we have enabled full TLS. For that you have to set the CAS_CLIENT_SSL_CA_LIST environment variable. Depending if you use the binary or the HTTP protocol you have to use a backend CA certficate or the Ingress CA certicate. Both can be donwloaded from https://github.com/sascommunities/hackinsas-2021/tree/main/CA-certificates  

In [2]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'ca_cert.pem'
s=swat.CAS('<team>-cas.hackinsas.vectorlabs.sas.com', 5570, 'viya_admin', '<your password>')

# or using REST 

#os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'star_hackinsas_vectorlabs_sas_com.pem'
#s=swat.CAS('https://<team>.hackinsas.vectorlabs.sas.com/cas-shared-default-http', username='viya_admin', password='<your password>')
#s=swat.CAS('<team>.hackinsas.vectorlabs.sas.com/cas-shared-default-http', 443, protocol='https', username='viya_admin', password='<your password>')



In [3]:
s.builtins.listNodes()

NOTE: Information is available on 1 nodes.


,name,role,connected,IP Address
0,controller.sas-cas-server-default.hackinsas-cr...,controller,Yes,10.244.6.5


In [9]:
s.serverStatus()

NOTE: Grid node action status report: 1 nodes, 8 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P01142021',
  'Viya Release': '20210128.1611850929670',
  'Viya Version': 'Stable 2020.1.2',
  'Copyright': 'Copyright © 2014-2020 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2021-02-04T10:39:03Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.hackinsas-teamapi.svc.cluster.local',
   'OS Name': 'Linux',
   'OS Family': 'LIN X64',
   'OS Release': '5.4.0-1035-azure',
   'OS Version': '#36~18.04.1-Ubuntu SMP Wed Dec 16 23:49:28 UTC 2020',
   'Model Number': 'x86_64',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.2 (Ootpa)'},
  'license': {'site': 'SAS INSTITUTE INC - GLOBAL HACKATHON 2021',
   'siteNum': 70281021,
   'expires': '30Apr2021:00:00:00',
   'gracePeriod': 45,
   'warningPeriod': 45},
  'CASHostAccountRequired': 'OPTIONAL'}

[server]

 Server Status
 
    nodes  actions
 0      1        8

[nodestatus]

 Node Status
 
                                                 name        role  uptime  running  stalled
 0  controller.sas-cas-server-default.hackinsas-te...  controller   7.534        0        0

+ Elapsed: 0.000685s, user: 0.000652s, mem: 0.304mb

In [6]:
s.sessionprop.setsessopt(caslib="PUBLIC") 

NOTE: 'Public' is now the active caslib.


+ Elapsed: 0.000384s, sys: 0.000366s, mem: 0.27mb

In [7]:
s.caslibinfo()

,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,CASUSER(viya_admin),PATH,Personal File System Caslib,/cas/data/caslibs/casuserlibraries/viya_admin/,,1.0,0.0,0.0,1.0,0.0,1.0
1,Formats,PATH,Stores user defined formats.,/cas/data/caslibs/formats/,,0.0,0.0,0.0,0.0,0.0,0.0
2,ModelPerformanceData,PATH,Stores performance data output for the Model M...,/cas/data/caslibs/modelMonitorLibrary/,,0.0,0.0,0.0,0.0,0.0,0.0
3,Models,PATH,Stores models created by Visual Analytics for ...,/cas/data/caslibs/models/,,0.0,0.0,0.0,0.0,0.0,0.0
4,Public,PATH,"Shared and writeable caslib, accessible to all...",/cas/data/caslibs/public/,,0.0,0.0,1.0,0.0,0.0,0.0
5,Samples,PATH,"Stores sample data, supplied by SAS.",/cas/data/caslibs/samples/,,0.0,0.0,0.0,0.0,0.0,0.0
6,SystemData,PATH,"Stores application generated data, used for ge...",/cas/data/caslibs/sysData/,,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
s.tableinfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,CAS_CARS,428,15,0,utf-8,2021-02-03T08:32:06+00:00,2021-02-03T08:33:07+00:00,2021-02-03T08:33:07+00:00,UTF8,1.927960e+09,...,0,0,0,,,0,viya_admin,,,NaN


In [ ]:
s.end